In [ ]:
def collect_data():
	name = input("Enter name of person : ")

	count = 0
	ids = input("Enter ID: ")

	cap = cv2.VideoCapture(0)

	filename = "haarcascade_frontalface_default.xml"

	cascade = cv2.CascadeClassifier(filename)

	while True:
		_, frm = cap.read()

		# gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)

		gray = preprocess_image(frm)

		faces = cascade.detectMultiScale(gray, 1.4, 1)

		for x,y,w,h in faces:
			cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
			roi = gray[y:y+h, x:x+w]

			cv2.imwrite(f"persons/{name}_{ids}_{count}.jpg", roi)
			count = count + 1
			cv2.putText(frm, f"{count}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
			cv2.imshow("new", roi)


		cv2.imshow("identify", frm)

		if cv2.waitKey(1) == 27 or count > 300:
			cv2.destroyAllWindows()
			cap.release()
			train()
			break

In [19]:
def identify():
	cap = cv2.VideoCapture(0)

	filename = "haarcascade_frontalface_default.xml"

	# paths = [os.path.join("persons", im) for im in os.listdir("persons")]
	# labelslist = {}
	# for path in paths:
	# 	labelslist[path.split('/')[-1].split('-')[2].split('.')[0]] = path.split('/')[-1].split('-')[0]

	paths = [os.path.join("persons", im) for im in os.listdir("persons")]
	labelslist = {}
	for path in paths:
		# Assuming the filename format is "name-id-count-etc.jpg"
		parts = path.split(os.sep)[-1].split('_')
		id_ = parts[1]  # ID is now in the second position
		name = parts[0]  # Name is still in the first position
		labelslist[id_] = name


	print(labelslist)
	recog = cv2.face.LBPHFaceRecognizer_create()
	detector = MTCNN()
 
	recog.read('model.yml')

	cascade = cv2.CascadeClassifier(filename)

	while True:
		_, frm = cap.read()

		# gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
  
		gray = preprocess_image(frm)


		# faces = cascade.detectMultiScale(gray, 1.3, 2)
  
		#mtcnn
		results = detector.detect_faces(frm)

		for face in results:
			x,y,w,h = face['box']
			cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
			roi = gray[y:y+h, x:x+w]

			label = recog.predict(roi)
   
			if label[1] < 100:
				print(label)
				cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
			else:
				cv2.putText(frm, "unkown", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

		cv2.imshow("identify", frm)

		if cv2.waitKey(1) == 27:
			cv2.destroyAllWindows()
			cap.release()
			break


In [ ]:
def mark_attendance1():              
    def upload_image():
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        
        file_path = filedialog.askopenfilename(title="Select an image file", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        
        if file_path:
            tkinter.messagebox.showinfo("Selected file path:", file_path)
            
            #predicting the collected file
            image = cv2.imread(file_path)
            
            gray = preprocess_image(image)
            
            results = detector.detect_faces(image)
            for face in results:
                x,y,w,h = face['box']
                cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]

                label = recog.predict(roi)
                print(label)
                
                if label[1] < 100:
                    cv2.putText(image, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                else:
                    cv2.putText(image, "unkown", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

            cv2.imshow("identify", image)
        else:
            tkinter.messagebox.showinfo("No file selected.")
        option_window.destroy()
        
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        att_file = 'attendance.xlsx'
        wb = load_workbook(att_file)
        ws = wb.active
        
        filename = "haarcascade_frontalface_default.xml"

        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()
            
            gray = preprocess_image(frm)
            faces = cascade.detectMultiScale(gray, 1.3, 2)

            for x,y,w,h in faces:
                cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                    ws.append([label[0],labelslist[str(label[0])],datetime.now().strftime("%H:%M:%S")])

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                wb.save(att_file)
                cv2.destroyAllWindows()
                cap.release()
                break
        option_window.destroy()
    
    # dataset = "sizeResized600"
    # paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]
    # labelslist = {}
    # for path in paths:
    #     # Assuming the filename format is "name-id-count-etc.jpg"
    #     parts = path.split(os.sep)[-1].split('_')
    #     id_ = parts[1]  # ID is now in the second position
    #     name = parts[0]  # Name is still in the first position
    #     labelslist[id_] = name
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)


    recog = cv2.face.LBPHFaceRecognizer_create()
    detector = MTCNN()

    recog.read('model600.yml')
    
    
    # Create a new window for user options
    option_window = tk.Tk()
    option_window.title("Recognition Options")
    option_window.geometry('720x540')
    option_window.config(bg='#607B9E')
    
    # Set font for the label
    label_font = font.Font(size=25, weight='bold', family='Helvetica')
    
    button_frame = tk.Frame(option_window, borderwidth=1, relief="groove", bg="#333333")
    button_frame.grid(row=2, column=1, padx=250, pady=175)   

    upload_button = tk.Button(button_frame, text="Upload Image", command=upload_image, width=20, height=2, bg="#e8de1a", font=label_font)
    upload_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    real_time_button = tk.Button(button_frame, text="Real-Time Recognition", command=real_time_recognition, width=20, height=2, bg="#e8de1a", font=label_font)
    real_time_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    option_window.mainloop()

In [6]:
def mark_attendance():
    def upload_image():
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        
        file_path = filedialog.askopenfilename(title="Select an image file", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        
        if file_path:
            tkinter.messagebox.showinfo("Selected file path:", file_path)
            
            #predicting the collected file
            image = cv2.imread(file_path)
            
            gray = preprocess_image(image)
            
            results = detector.detect_faces(image)
            for face in results:
                x,y,w,h = face['box']
                cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]

                label = recog.predict(roi)
                print(label)
                
                if label[1] < 100:
                    cv2.putText(image, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                else:
                    cv2.putText(image, "unkown", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

            cv2.imshow("identify", image)
            
            
            filename = "haarcascade_frontalface_default.xml"

            # paths = [os.path.join("persons", im) for im in os.listdir("persons")]
            # labelslist = {}
            # for path in paths:
            # 	labelslist[path.split('/')[-1].split('-')[2].split('.')[0]] = path.split('/')[-1].split('-')[0]


            # cascade = cv2.CascadeClassifier(filename)
            # faces =  cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10)
            # for x,y,w,h in faces:
            #     cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
            #     roi = gray[y:y+h, x:x+w]
            #     roi = cv2.resize(roi, dsize=(160, 160))
                
            #     label = recog.predict(roi)
            #     print(label)
                
            #     if label[1] < 100:
            #         cv2.putText(image, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
            #     else:
            #         cv2.putText(image, "unkown", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

            # cv2.imshow("identify", image)
        else:
            tkinter.messagebox.showinfo("No file selected.")
        option_window.destroy()
        
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        # cap = cv2.VideoCapture(f'http://192.168.202.85:8080/video')

        filename = "haarcascade_frontalface_default.xml"

        # paths = [os.path.join("persons", im) for im in os.listdir("persons")]
        # labelslist = {}
        # for path in paths:
        # 	labelslist[path.split('/')[-1].split('-')[2].split('.')[0]] = path.split('/')[-1].split('-')[0]


        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()

            # gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
    
            gray = preprocess_image(frm)


            faces = cascade.detectMultiScale(gray, 1.3, 2)
    
            #mtcnn
            # results = detector.detect_faces(frm)
            #print(faces)
            for x,y,w,h in faces:
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                # else:
                #     cv2.putText(frm, "unkown", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                cv2.destroyAllWindows()
                cap.release()
                break
        option_window.destroy()
    
    # dataset = "sizeResized600"
    # paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]
    # labelslist = {}
    # for path in paths:
    #     # Assuming the filename format is "name-id-count-etc.jpg"
    #     parts = path.split(os.sep)[-1].split('_')
    #     id_ = parts[1]  # ID is now in the second position
    #     name = parts[0]  # Name is still in the first position
    #     labelslist[id_] = name
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)


    recog = cv2.face.LBPHFaceRecognizer_create()
    detector = MTCNN()

    recog.read('model600.yml')
    
    
    # Create a new window for user options
    option_window = tk.Tk()
    option_window.title("Recognition Options")
    option_window.geometry('720x540')
    option_window.config(bg='#607B9E')
    
    # Set font for the label
    label_font = font.Font(size=25, weight='bold', family='Helvetica')
    
    button_frame = tk.Frame(option_window, borderwidth=1, relief="groove", bg="#333333")
    button_frame.grid(row=2, column=1, padx=250, pady=175)   

    upload_button = tk.Button(button_frame, text="Upload Image", command=upload_image, width=20, height=2, bg="#e8de1a", font=label_font)
    upload_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    real_time_button = tk.Button(button_frame, text="Real-Time Recognition", command=real_time_recognition, width=20, height=2, bg="#e8de1a", font=label_font)
    real_time_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    option_window.mainloop()
mark_attendance()

{'1': 'Sunitha mam', '14': 'Ashish', '20': 'Chetan', '33': 'Kushal', '34': 'Lakshmi', '49': 'Muzammil', '54': 'NIthish', '59': 'Pavankumar', '78': 'Sanjana', '80': 'Shamant', '83': 'Shivani', '89': 'Suhas', '405': 'Umar'}
(49, 77.64240856561534)
(49, 76.03745224495685)
(49, 77.82138091048135)
(49, 76.68940396247211)
(89, 71.54308802376033)
(89, 70.05417906634352)
(89, 70.1671654097192)
(89, 68.28890505373738)
(89, 72.24148647690389)
(49, 80.19943050435145)
(49, 79.96716094550243)
(49, 81.88930328430381)
(54, 82.74213379506963)
(49, 79.25560858382815)
(49, 77.31404284863365)
(49, 75.74839971395782)
(49, 79.11655612423927)
(49, 74.95294128027852)
(49, 76.54269070483731)
(49, 73.72773675203123)
(49, 74.4393569182605)
(49, 74.51256251231302)
(49, 75.37315268095324)
(49, 72.85311689163359)
(54, 76.6983793430311)
(54, 78.04247808076893)
(49, 76.53141929123002)
(49, 74.35661571703307)


In [3]:
def mark_attendance2():              
    def upload_image():
        root = tk.Tk()
        root.withdraw()  # Hide the main window
        
        file_path = filedialog.askopenfilename(title="Select an image file", filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        
        if file_path:
            tkinter.messagebox.showinfo("Selected file path:", file_path)
            
            #predicting the collected file
            image = cv2.imread(file_path)
            
            gray = preprocess_image(image)
            
            results = detector.detect_faces(image)
            for face in results:
                x,y,w,h = face['box']
                cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]

                label = recog.predict(roi)
                print(label)
                
                if label[1] < 100:
                    cv2.putText(image, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

            cv2.imshow("identify", image)
        else:
            tkinter.messagebox.showinfo("No file selected.")
        option_window.destroy()
        
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        att_file = 'attendance.xlsx'
        wb = load_workbook(att_file)
        ws = wb.active
        
        preds = []
        
        filename = "haarcascade_frontalface_default.xml"

        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()
            
            gray = preprocess_image(frm)
            faces = cascade.detectMultiScale(gray, 1.3, 2)

            for x,y,w,h in faces:
                cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                cv2.putText(frm, f"{len(preds)}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                    preds.append(label[0])
                    if  len(preds) >= 20:
                        pred_dict = {f"{pred}":preds.count(pred) for pred in set(preds)}
                        max = 0
                        crct_pred = None
                        for  k,v in pred_dict.items():
                            if max < v:
                                max = v
                                crct_pred = k
                        print(preds,crct_pred,sep="\n")
                        ws.append([int(crct_pred),labelslist[crct_pred],datetime.now().strftime("%H:%M:%S")])
                        preds.clear()

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                wb.save(att_file)
                cv2.destroyAllWindows()
                cap.release()
                break
        option_window.destroy()
    
    # dataset = "sizeResized600"
    # paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]
    # labelslist = {}
    # for path in paths:
    #     # Assuming the filename format is "name-id-count-etc.jpg"
    #     parts = path.split(os.sep)[-1].split('_')
    #     id_ = parts[1]  # ID is now in the second position
    #     name = parts[0]  # Name is still in the first position
    #     labelslist[id_] = name
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)


    recog = cv2.face.LBPHFaceRecognizer_create()
    detector = MTCNN()

    recog.read('model600.yml')
    
    
    # Create a new window for user options
    option_window = tk.Tk()
    option_window.title("Recognition Options")
    option_window.geometry('720x540')
    option_window.config(bg='#607B9E')
    
    # Set font for the label
    label_font = font.Font(size=25, weight='bold', family='Helvetica')
    
    button_frame = tk.Frame(option_window, borderwidth=1, relief="groove", bg="#333333")
    button_frame.grid(row=2, column=1, padx=250, pady=175)   

    upload_button = tk.Button(button_frame, text="Upload Image", command=upload_image, width=20, height=2, bg="#e8de1a", font=label_font)
    upload_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    real_time_button = tk.Button(button_frame, text="Real-Time Recognition", command=real_time_recognition, width=20, height=2, bg="#e8de1a", font=label_font)
    real_time_button.pack(side="top", pady=10, padx=10)  # Centered vertically with padding

    option_window.mainloop()

In [ ]:
def mark_attendance3():                  
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        att_file = 'attendance.xlsx'
        wb = load_workbook(att_file)
        ws = wb.active
        
        preds = []
        
        filename = "haarcascade_frontalface_default.xml"

        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()
            
            gray = preprocess_image(frm)
            faces = cascade.detectMultiScale(gray, 1.3, 2)

            for x,y,w,h in faces:
                cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                cv2.putText(frm, f"{len(preds)}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                    preds.append(label[0])
                    if  len(preds) >= 20:
                        pred_dict = {f"{pred}":preds.count(pred) for pred in set(preds)}
                        max = 0
                        crct_pred = None
                        for  k,v in pred_dict.items():
                            if max < v:
                                max = v
                                crct_pred = k
                        print(preds,crct_pred,sep="\n")
                        ws.append([int(crct_pred),labelslist[crct_pred],datetime.now().strftime("%H:%M:%S")])
                        preds.clear()

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                wb.save(att_file)
                cv2.destroyAllWindows()
                cap.release()
                break
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)

    recog = cv2.face.LBPHFaceRecognizer_create()

    recog.read('model600.yml')
    real_time_recognition()

In [5]:
import cv2
def preprocess_image(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    eq = cv2.equalizeHist(blurred)
    return eq

In [2]:
import tkinter as tk

def get_info():
  """Retrieves user input from the Entry widgets and stores them in variables."""
  global name, user_id
  name = name_entry.get()
  user_id = id_entry.get()

  # You can add further processing or validation of the retrieved data here

  # Optional: Clear entry fields after submission (uncomment if desired)
  # name_entry.delete(0, tk.END)
  # id_entry.delete(0, tk.END)

  # Display a confirmation message (uncomment if desired)
  # message = f"Name: {name}\nID: {user_id}"
  # info_label.config(text=message)
  print(name,id)

# Create the main window
root = tk.Tk()
root.title("Name and ID Collector")

# Set window geometry and background color
root.geometry("400x200")  # Adjust width and height as desired
root.configure(background="lightblue")  # Change background color to your preference

# Create labels for name and ID
name_label = tk.Label(root, text="Name:")
name_label.pack()

id_label = tk.Label(root, text="ID:")
id_label.pack()

# Create Entry widgets for user input, placing them vertically
name_entry = tk.Entry(root)
name_entry.pack()

id_entry = tk.Entry(root)
id_entry.pack()

# Create a button to trigger the get_info function, placed below entries
submit_button = tk.Button(root, text="Submit", command=get_info)
submit_button.pack()

# Optional: Add a label to display information after submission (uncomment if desired)
# info_label = tk.Label(root, text="")
# info_label.pack()

# Start the main event loop
root.mainloop()


Muzammil <built-in function id>


In [4]:
import tkinter as tk

def get_info_and_destroy():
  """Retrieves user input, stores in variables, and destroys the window."""
  global name, user_id
  name = name_entry.get()
  user_id = id_entry.get()

  # You can add further processing or validation of the retrieved data here

  # Optional: Clear entry fields after submission (uncomment if desired)
  # name_entry.delete(0, tk.END)
  # id_entry.delete(0, tk.END)

  # Optional: Display a confirmation message before destroying (uncomment if desired)
  # message = f"Name: {name}\nID: {user_id}"
  # info_label.config(text=message)

  root.destroy()  # Destroy the window after retrieving data

# Create the main window
root = tk.Tk()
root.title("Name and ID Collector")

# Set window geometry and background color
root.geometry("400x200")  # Adjust width and height as desired
root.configure(background="lightblue")  # Change background color to your preference

# Create labels for name and ID
name_label = tk.Label(root, text="Name:")
name_label.pack()

# Create Entry widgets for user input, placing them vertically
name_entry = tk.Entry(root)
name_entry.pack()

id_label = tk.Label(root, text="ID:")
id_label.pack()

id_entry = tk.Entry(root)
id_entry.pack()

# Create a button to trigger get_info_and_destroy, placed below entries
submit_button = tk.Button(root, text="Submit", command=get_info_and_destroy)
submit_button.pack()

# Global variables to store user input (declared outside functions)
name = ""
user_id = ""

# Optional: Add a label to display information after submission (uncomment if desired)
# info_label = tk.Label(root, text="")
# info_label.pack()

# Start the main event loop
root.mainloop()


In [7]:
import tkinter as tk

def get_info_and_destroy():
  """Retrieves user input, stores in variables, and destroys the window."""
  global name, user_id
  name = name_entry.get()
  user_id = id_entry.get()

  # You can add further processing or validation of the retrieved data here

  # Optional: Clear entry fields after submission (uncomment if desired)
  # name_entry.delete(0, tk.END)
  # id_entry.delete(0, tk.END)

  # Optional: Display a confirmation message before destroying (uncomment if desired)
  # message = f"Name: {name}\nID: {user_id}"
  # info_label.config(text=message)

  root.destroy()  # Destroy the window after retrieving 
  print(name,user_id)

# Create the main window
root = tk.Tk()
root.title("Name and ID Collector")

# Set window geometry and background color
root.geometry("400x200")  # Adjust width and height as desired
root.configure(background="lightblue")  # Change background color to your preference

# Create a frame to hold labels and entries, centered in the window
frame = tk.Frame(root)
frame.pack(expand=True)

# Create labels for name and ID, placed on the left side of the frame
name_label = tk.Label(frame, text="Name:")
name_label.grid(row=0, column=0, sticky="w")

id_label = tk.Label(frame, text="ID:")
id_label.grid(row=1, column=0, sticky="w")

# Create Entry widgets for user input, placed on the right side of the frame
name_entry = tk.Entry(frame)
name_entry.grid(row=0, column=1, padx=10)  # Add padding between label and entry

id_entry = tk.Entry(frame)
id_entry.grid(row=1, column=1, padx=10)  # Add padding between label and entry

# Create a button to trigger get_info_and_destroy, placed below entries in the frame
submit_button = tk.Button(frame, text="Submit", command=get_info_and_destroy)
submit_button.grid(row=2, columnspan=2, pady=10)  # Span across both columns and add padding

# Global variables to store user input (declared outside functions)
name = ""
user_id = ""

# Start the main event loop
root.mainloop()


muzammil 49


In [65]:
import tkinter as tk
import cv2
import main

def register():
    """
    Collects user input for name and ID through a Tkinter GUI,
    processes and stores captured face images using OpenCV.

    Displays a separate window for face detection and image capture.
    """
    def get_info_and_destroy():
        """Retrieves user input from the Entry widgets and stores them in variables."""
        name = name_entry.get()
        id = int(id_entry.get())

        # Validate or process name and ID here if needed
        if not name or not id:
            # Display an error message
            print("Enter valid name and id")
            return

        print(name,id)
        # Capture face images
        capture_faces(name, id)

        root.destroy()  # Destroy the input window

    # Create the main window for name and ID collection
    root = tk.Tk()
    root.title("Student Registration")

    # Set window geometry and background color
    root.geometry("480x240")  # Adjust width and height as desired
    root.config(bg='#607B9E')

    # Create a frame to hold labels and entries, centered in the window
    frame = tk.Frame(root, bg="#333333")
    frame.pack(expand=True)

    # Create labels for name and ID, placed on the left side of the frame
    name_label = tk.Label(frame, text="Name:", width=6, height=1, bg="#e8de1a")
    name_label.grid(row=0, column=0, sticky="w", padx=(10,5), pady=(10,5))

    id_label = tk.Label(frame, text="ID:",width=6, height=1, bg="#e8de1a",)
    id_label.grid(row=1, column=0, sticky="w", padx=(10,5), pady=(5,5))

    # Create Entry widgets for user input, placed on the right side of the frame
    name_entry = tk.Entry(frame, bg="#e8de1a")
    name_entry.grid(row=0, column=1, padx=(5,10), pady=(10,5))  # Add padding between label and entry

    id_entry = tk.Entry(frame, bg="#e8de1a")
    id_entry.grid(row=1, column=1, padx=(5,10), pady=(5,5))  # Add padding between label and entry

    # Create a button to trigger get_info_and_destroy, placed below entries in the frame
    submit_button = tk.Button(frame, text="Submit", command=get_info_and_destroy, bg="#e8de1a", height=1)
    submit_button.grid(row=2, columnspan=2, pady=5)  # Span across both columns and add padding

    root.mainloop()  # Start the event loop for the input window

def capture_faces(name, id):
    """Captures face images using OpenCV."""

    count = 0

    cap = cv2.VideoCapture(0)

    filename = "haarcascade_frontalface_default.xml"

    cascade = cv2.CascadeClassifier(filename)

    while True:
        _, frm = cap.read()

        # gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)

        gray = preprocess_image(frm)

        faces = cascade.detectMultiScale(gray, 1.4, 1)

        for x,y,w,h in faces:
            cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
            roi = gray[y:y+h, x:x+w]

            #cv2.imwrite(f"persons/{name}_{id}_{count}.jpg", roi)
            count = count + 1
            cv2.putText(frm, f"{count}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
            cv2.imshow("new", roi)


        cv2.imshow("identify", frm)

        if cv2.waitKey(1) == 27 or count > 300:
            cv2.destroyAllWindows()
            cap.release()
            break

In [66]:
register()

In [ ]:
import pandas as pd
df = pd.read_excel('names_ids.xlsx')
# names= df['Name'].to_list()
# ids = df['ID'].to_list()
# print(names,ids,sep="\n")
df.drop_duplicates(keep='last',inplace=True)
print(df)

In [21]:
students_list = dict(zip(ids,names))

In [1]:
import os
import shutil
from PIL import Image, ImageEnhance
import tkinter as tk
import tkinter.font as font
def preprocess_and_save(original_folder):
    """
    Processes all images in the original folder and applies a series of transformations
    to each image, saving the results in the output folder.
    Saves both flipped and unflipped versions of each image, with and without noise.
    """
    # Create output folder if it doesn't exist
    output_folder = "E:/contiClone/persons"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Define the 'raw' folder path
    raw_folder = original_folder
    if not os.path.exists(raw_folder):
        os.makedirs(raw_folder)

    for filename in os.listdir(original_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(original_folder, filename)
            with Image.open(image_path) as img:
                base_name = os.path.splitext(os.path.basename(image_path))[0]
                
                # Define transformation parameters
                brightness_levels = [0.7, 1.0, 1.2]
                contrast_levels = [0.7, 1.0, 1.2]
                rotation_angles = [-5, 0, 5]
                
                # Apply transformations
                for brightness in brightness_levels:
                    bright_img = ImageEnhance.Brightness(img).enhance(brightness)
                    for contrast in contrast_levels:
                        contrast_img = ImageEnhance.Contrast(bright_img).enhance(contrast)
                        for angle in rotation_angles:
                            rotated_img = contrast_img.rotate(angle)
                            original_filename = f"{base_name}_b{brightness}_c{contrast}_r{angle}.jpg"
                            rotated_img.save(os.path.join(output_folder, original_filename))
                            
                            
                            flipped_img = rotated_img.transpose(Image.FLIP_LEFT_RIGHT)
                            flipped_filename = f"{base_name}_b{brightness}_c{contrast}_r{angle}_flipped.jpg"
                            flipped_img.save(os.path.join(output_folder, flipped_filename))
                            
                            
            # Move the original image to the 'raw' folder
            shutil.move(image_path, os.path.join(raw_folder, filename))
preprocess_and_save("C:/Users/passi/OneDrive/Desktop/Final_year_project/models/lbph/Dataset2/raw")

In [1]:
from PIL import Image
import os

def resize_images(input_folder, output_folder, target_size):
  """
  Resizes all images in a folder to a target size and saves the resized copies in an output folder.

  Args:
      input_folder (str): Path to the folder containing the images to resize.
      output_folder (str): Path to the folder where the resized images will be saved.
      target_size (tuple): A tuple representing the target width and height (e.g., (600, 400)).
  """

  # Create the output folder if it doesn't exist
  os.makedirs(output_folder, exist_ok=True)

  # Loop through all files in the input folder
  for filename in os.listdir(input_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
      # Get the full path of the image
      image_path = os.path.join(input_folder, filename)

      try:
        # Open the image using Pillow
        img = Image.open(image_path)

        # # Preserve aspect ratio while resizing
        # width, height = img.size
        # aspect_ratio = width / height
        # new_width = target_size[0]
        # new_height = int(new_width / aspect_ratio)

        # if new_height > target_size[1]:
        #     # Adjust height if it exceeds target
        #     new_height = target_size[1]
        #     new_width = int(new_height * aspect_ratio)

        # Resize the image with high-quality resampling (LANCZOS)
        resized_img = img.resize(target_size, Image.LANCZOS)


        # Save the resized image in the output folder
        resized_img.save(os.path.join(output_folder, filename))
        print(f"Resized {filename}")
      except OSError as e:
        print(f"Error processing '{filename}': {e}")

# Example usage
input_folder = "size600"
output_folder = "sizeResized600"
target_size = (160, 160)  # Target width and height

resize_images(input_folder, output_folder, target_size)


Resized Ashish_14_0.jpg
Resized Ashish_14_1.jpg
Resized Ashish_14_10.jpg
Resized Ashish_14_100.jpg
Resized Ashish_14_101.jpg
Resized Ashish_14_102.jpg
Resized Ashish_14_103.jpg
Resized Ashish_14_104.jpg
Resized Ashish_14_105.jpg
Resized Ashish_14_106.jpg
Resized Ashish_14_107.jpg
Resized Ashish_14_108.jpg
Resized Ashish_14_109.jpg
Resized Ashish_14_11.jpg
Resized Ashish_14_110.jpg
Resized Ashish_14_111.jpg
Resized Ashish_14_112.jpg
Resized Ashish_14_113.jpg
Resized Ashish_14_114.jpg
Resized Ashish_14_115.jpg
Resized Ashish_14_116.jpg
Resized Ashish_14_117.jpg
Resized Ashish_14_118.jpg
Resized Ashish_14_119.jpg
Resized Ashish_14_12.jpg
Resized Ashish_14_120.jpg
Resized Ashish_14_121.jpg
Resized Ashish_14_122.jpg
Resized Ashish_14_124.jpg
Resized Ashish_14_125.jpg
Resized Ashish_14_126.jpg
Resized Ashish_14_127.jpg
Resized Ashish_14_128.jpg
Resized Ashish_14_129.jpg
Resized Ashish_14_13.jpg
Resized Ashish_14_130.jpg
Resized Ashish_14_131.jpg
Resized Ashish_14_132.jpg
Resized Ashish_14_13

In [ ]:
df = pd.read_excel('names_ids.xlsx')
names = list(df['Name'])
ids = list(df['ID'])

labelslist = {}
for name, id in zip(names, ids):
    # Convert the id to a string
    id = str(id)
    # Add the id:name pair to the dictionary
    labelslist[id] = name
print(labelslist)

In [ ]:
import pandas as pd

# Function to mark attendance in an Excel file
def mark_attendance_in_excel(student_id, student_name, attendance_time):
    # Load the existing attendance data
    attendance_data = pd.read_excel("attendance.xlsx")
    
    # Add new attendance record
    new_record = {"ID": student_id, "Name": student_name, "Time": attendance_time}
    attendance_data = attendance_data.append(new_record, ignore_index=True)
    
    # Save the updated attendance data
    attendance_data.to_excel("attendance.xlsx", index=False)

# Function to detect students and mark attendance
def real_time_recognition():
    # ... (Existing code for detecting students)

    # Predict the student's ID
    label = recog.predict(roi)[0]

    # Get the student's name from the labelslist
    student_name = labelslist[str(label)]

    # Get the current time
    attendance_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Mark attendance in the Excel file
    mark_attendance_in_excel(label, student_name, attendance_time)

    # ... (Existing code for displaying the student's name)

In [3]:
from datetime import datetime
import pandas as pd
def mark_attendance_in_excel(student_id, student_name, attendance_time):
        # Load the existing attendance data
        attendance_data = pd.read_excel("attendance.xlsx")
        
        # Add new attendance record
        new_record = {"ID": student_id, "Name": student_name, "Time": attendance_time}
        attendance_data = attendance_data.append(new_record, ignore_index=True)
        
        # Save the updated attendance data
        attendance_data.to_excel("attendance.xlsx", index=False)
mark_attendance_in_excel("14", "John Doe", datetime.now().strftime("%H:%M:%S"))

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import openpyxl

# Replace with your file path and data list
file_path = "attendance.xlsx"
data_to_append = ["Data 1", "Data 2", "Data 3"]

# Open the workbook
workbook = openpyxl.load_workbook(file_path)

# Get the active sheet (or specify the sheet name)
sheet = workbook.active

# Get the last row number
last_row = sheet.max_row

# Create a new row at the end + 1
new_row = sheet.create_row(last_row + 1)

# Assign data to each cell in the new row
for col, value in enumerate(data_to_append):
  new_row[col].value = value

# Save the workbook
workbook.save(file_path)

print("Data appended successfully!")


In [8]:
import pandas as pd

# Existing Excel file
existing_file = 'attendance.xlsx'

# New data to append
new_data = {"ID": [49], "Name": ["muzammil"]}
df_new = pd.DataFrame(new_data)

# Read existing data
df_existing = pd.read_excel(existing_file)
print(df_existing)
# Append new data
df_combined = df_existing.append(df_new, ignore_index=True)

print(df_combined)
# Save the combined data to Excel
df_combined.to_excel(existing_file, index=False)


   ID    Name      Time
0  14  Ashish  13:15:45


AttributeError: 'DataFrame' object has no attribute 'append'

In [10]:
from openpyxl import load_workbook

# Existing Excel file
existing_file = 'attendance.xlsx'

# New data to append
new_data = [[49,"muzammil",datetime.now().strftime("%H:%M:%S")]]

# Load existing workbook
wb = load_workbook(existing_file)

# Select the active sheet
ws = wb.active

# Append new data
for row in new_data:
	ws.append(row)

# Save the workbook
wb.save(existing_file)


In [1]:
preds = [15,14,14,15,49,49,30,49,49,30]
if  len(preds) >= 0:
    pred_dict = {f"{pred}":preds.count(pred) for pred in set(preds)}
    max = 0
    crct_pred = None
    for  k,v in pred_dict.items():
        if max < v:
            max = v
            crct_pred = k
print(crct_pred)
print(pred_dict)

49
{'49': 4, '30': 2, '14': 2, '15': 2}


In [10]:
if  len(preds) >= 10:
    max = 0
    crct_pred = None
    for  k,v in pred_dict.items():
        if max < v:
            max = v
            crct_pred = str(k)
    print(preds,crct_pred,sep="\n")
print(str(crct_pred))
print("hello")

[49, 15, 14, 14, 15, 49, 30, 49, 49, 30]
49
49
hello


In [ ]:
def mark_attendance3():  
    def get_date():
        """Retrieves user input from the Entry widgets and stores them in variables."""
        name = name_entry.get()
        id = id_entry.get()

        # Validate or process name and ID here if needed
        if not name or not id:
            # Display an error message
            print("Enter valid name and id")
            return
        
        # Capture face images
        capture_faces(name, id)

        root.destroy()  # Destroy the input window                
    def real_time_recognition():
        cap = cv2.VideoCapture(0)
        att_file = 'attendance.xlsx'
        wb = load_workbook(att_file)
        ws = wb.active
        
        preds = []
        
        filename = "haarcascade_frontalface_default.xml"

        cascade = cv2.CascadeClassifier(filename)

        while True:
            _, frm = cap.read()
            
            gray = preprocess_image(frm)
            faces = cascade.detectMultiScale(gray, 1.3, 2)

            for x,y,w,h in faces:
                cv2.rectangle(frm, (x,y), (x+w, y+h), (0,255,0), 2)
                roi = gray[y:y+h, x:x+w]
                roi = cv2.resize(roi, dsize=(160, 160))
                
                cv2.putText(frm, f"{len(preds)}", (20,20), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3)
                label = recog.predict(roi)
    
                if label[1] < 100:
                    print(label)
                    cv2.putText(frm, f"{labelslist[str(label[0])]}", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                    preds.append(label[0])
                    if  len(preds) >= 20:
                        pred_dict = {f"{pred}":preds.count(pred) for pred in set(preds)}
                        max = 0
                        crct_pred = None
                        for  k,v in pred_dict.items():
                            if max < v:
                                max = v
                                crct_pred = k
                        print(preds,crct_pred,sep="\n")
                        ws.append([int(crct_pred),labelslist[crct_pred],datetime.now().strftime("%H:%M:%S")])
                        preds.clear()

            cv2.imshow("identify", frm)

            if cv2.waitKey(1) == 27:
                wb.save(att_file)
                cv2.destroyAllWindows()
                cap.release()
                break
    
    df = pd.read_excel('names_ids.xlsx')
    names = list(df['Name'])
    ids = list(df['ID'])

    labelslist = {}
    for name, id in zip(names, ids):
        id = str(id)
        labelslist[id] = name
    print(labelslist)

    recog = cv2.face.LBPHFaceRecognizer_create()

    recog.read('model600.yml')
    real_time_recognition()

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
import tkinter.messagebox
from tkinter import filedialog
import tkinter as tk
import tkinter.font as font
from mtcnn.mtcnn  import MTCNN
from datetime import datetime
from openpyxl import load_workbook

In [14]:
window = tk.Tk()
window.title("AUTOMATED ATTENDANCE MANAGEMENT SYSTEM")

window.geometry('1080x720')
window.config(bg='#607B9E')


# Create a frame for the buttons
button_frame = tk.Frame(window, borderwidth=1, relief="groove", bg="#333333")
button_frame.grid(row=2, column=0, padx=100, pady=50)

# Set font for the label
label_font = font.Font(size=25, weight='bold', family='Helvetica')

label_title = tk.Label(window, text="ATME College of Engineering", bg="#e8de1a", font=label_font)
label_title.grid(row=0, column=0, columnspan=1, padx=(20, 20), pady=(60, 5)), 

# Create and position the label
label_title = tk.Label(window, text="Automated Attendance Management System", bg="#e8de1a", font=label_font)
label_title.grid(row=1, column=0, columnspan=1, padx=(150,150), pady=(40, 40))

# Create and position the buttons within the frame
register = tk.Button(button_frame, text="REGISTER", width=13, height=2, bg="#e8de1a", command=None, font=label_font)
register.grid(row=1, column=0, padx=8, pady=8)

attendance = tk.Button(button_frame, text="ATTENDANCE", width=13, height=2, bg="#e8de1a", command=None, font=label_font)
attendance.grid(row=1, column=1, padx=8, pady=8)

students = tk.Button(button_frame, text="STUDENTS", width=13, height=2, bg="#e8de1a", command=None, font=label_font)
students.grid(row=2, column=0, padx=8, pady=8)

report = tk.Button(button_frame, text="REPORT", width=13, height=2, bg="#e8de1a", command=None, font=label_font)
report.grid(row=2, column=1, padx=8, pady=8)

# Center the button frame horizontally using `sticky`
# button_frame.grid(columnspan=1,padx=(60,60), pady=(120,120) ,sticky="w")  # Adjust here


window.bind("<Escape>", lambda event: window.destroy())

window.mainloop()

In [1]:
import openpyxl
from datetime import datetime

def edit_excel_cell_by_value(filename, row_value, column_value, new_value):
    """
    Loads an Excel file, edits a cell based on matching row and column values,
    and saves the changes.

    Args:
        filename (str): The name of the Excel file.
        row_value (any): The value to search for in the row.
        column_value (any): The value to search for in the column.
        new_value (any): The new value to write to the cell.
    """

    try:
        # Load the Excel workbook
        wb = openpyxl.load_workbook(filename)
        sheet = wb.active  # Get the active worksheet

        # Find the cell with matching row and column values
        found = False
        for row in sheet.iter_rows():
            for cell in row:
                if cell.value == row_value and sheet.cell(row=cell.row, column=1).value == column_value:
                    found = True
                    target_cell = cell
                    break
            if found:
                break

        # Check if the cell was found
        if not found:
            print(f"Cell not found with row value '{row_value}' and column value '{column_value}'.")
            return

        # Edit the cell value
        target_cell.value = new_value

        # Save the changes to the same file
        wb.save(filename)

        print(f"Cell edited successfully! New value: {new_value}")

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")

# Example usage
filename = "students.xlsx"  # Replace with your actual filename
row_value = "49"  # Replace with your search value
column_value = "2024-05-07"  # Replace with your search value
new_value = datetime.now().strftime("%H:%M:%S")  # Replace with the new value

edit_excel_cell_by_value(filename, row_value, column_value, new_value)


Cell not found with row value '49' and column value '2024-05-07'.


In [3]:
import openpyxl

def edit_cell_by_id_and_date(filename, id_value, new_value):
    """
    Loads an Excel file, edits a cell based on matching ID and date values,
    and saves the changes.

    Args:
        filename (str): The name of the Excel file.
        id_value (any): The value to search for in the ID column.
        new_value (any): The new value to write to the cell.
    """

    try:
        # Load the Excel workbook
        wb = openpyxl.load_workbook(filename)
        sheet = wb.active  # Get the active worksheet

        # Find the target cell (assuming 'date' is in column C)
        found = False
        for row in sheet.iter_rows(min_row=2):  # Skip the header row (row 1)
            if row[2].value == id_value and row[2].value == "07-05-2024":  # Check ID and date
                found = True
                target_cell = row[0]  # Edit the first column (name)
                break

        # Check if the cell was found
        if not found:
            print(f"Cell not found with ID '{id_value}' and date '2024-05-07'.")
            return

        # Edit the cell value
        target_cell.value = new_value

        # Save the changes to the same file
        wb.save(filename)

        print(f"Cell edited successfully! New value for ID '{id_value}': {new_value}")

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")

# Example usage
filename = "students.xlsx"  # Replace with your actual filename
id_value = "49"  # Replace with the ID you want to match
new_value = datetime.now().strftime("%H:%M:%S")  # Replace with the new value to set

edit_cell_by_id_and_date(filename, id_value, new_value)


Cell not found with ID '49' and date '2024-05-07'.


In [16]:
import pandas as pd

# Specify the file path to your Excel file
file_path = 'students.xlsx'

# Prompt the user to enter the ID
id_to_mark = input("Enter the ID to mark attendance: ")

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)



In [17]:
df

,Name,ID,Unnamed: 2,2024-05-08
0,Ashish,14,NaN,NaN
1,Chetan,20,NaN,NaN
2,Lakshmi,34,NaN,NaN
3,Muzammil,49,NaN,\n15:00:56
4,NIthish,54,NaN,NaN
5,Pavankumar,59,NaN,NaN
6,Sanjana,78,NaN,NaN
7,Shivani,83,NaN,NaN
8,Suhas,89,NaN,NaN
9,Umar,405,NaN,NaN


In [7]:
df["ID"] == int(id_to_mark)

0    False
1    False
2    False
3     True
4    False
5    False
6    False
7    False
8    False
9    False
Name: ID, dtype: bool

In [20]:
import pandas as pd
from datetime import datetime

# Load the Excel file into a Pandas DataFrame
file_path = 'students.xlsx'
df = pd.read_excel(file_path)

# Get today's date in the format 'YYYY-MM-DD'
date_today = datetime.now().strftime("%Y-%m-%d")

# Check if today's date column exists, if not, add it
if date_today not in df.columns:
    df[date_today] = ''

# Update the attendance for the specific label
label = 49  # Assuming labelslist is defined
df.loc[df['ID'] == label, date_today] = df.loc[df['ID'] == label, date_today].astype(str) + ',  ' + datetime.now().strftime("%H:%M:%S")

# Save the updated DataFrame back to the Excel file
df.to_excel(file_path, index=False)

print("Attendance marked for today.")

Attendance marked for today.


In [12]:
type(df["ID"][3])

numpy.int64

In [21]:
df

,Name,ID,2024-05-08
0,Ashish,14,NaN
1,Chetan,20,NaN
2,Lakshmi,34,NaN
3,Muzammil,49,"15:00:56, 15:02:44, 15:04:07"
4,NIthish,54,NaN
5,Pavankumar,59,NaN
6,Sanjana,78,NaN
7,Shivani,83,NaN
8,Suhas,89,NaN
9,Umar,405,NaN


In [22]:
date_today not in df.columns

False

In [29]:
import numpy as np
df["hello"] = np.nan
df

,Name,ID,2024-05-08,hello
0,Ashish,14,NaN,NaN
1,Chetan,20,NaN,NaN
2,Lakshmi,34,NaN,NaN
3,Muzammil,49,"15:00:56, 15:02:44, 15:04:07",NaN
4,NIthish,54,NaN,NaN
5,Pavankumar,59,NaN,NaN
6,Sanjana,78,NaN,NaN
7,Shivani,83,NaN,NaN
8,Suhas,89,NaN,NaN
9,Umar,405,NaN,NaN


In [1]:
import pandas as pd

# Specify the file path to your Excel file
file_path = 'names_ids.xlsx'


# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)
df

,Name,ID
0,Ashish,14
1,Chetan,20
2,Lakshmi,34
3,Muzammil,49
4,NIthish,54
5,Pavankumar,59
6,Sanjana,78
7,Shivani,83
8,Suhas,89
9,Umar,405


In [5]:
df2 = df.copy()
df2 = df2.drop(9)
df2

,Name,ID
0,Ashish,14
1,Chetan,20
2,Lakshmi,34
3,Muzammil,49
4,NIthish,54
5,Pavankumar,59
6,Sanjana,78
7,Shivani,83
8,Suhas,89


In [6]:
df3 = df + (df-df2)
df3

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [18]:
from tkinter import ttk

def display_attendance():
    # Read student names and IDs from an Excel file
    df =pd.read_excel("students.xlsx")
    date_today = datetime.now().strftime("%Y-%m-%d")
    df = df[["Name","ID",date_today]]

    root = tk.Tk()
    root.title("Attendance List")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = tk.Frame(canvas, width=400, height=300)  # Set minimum size for the frame

    # Configure the canvas and scrollbar
    def update_scrollregion(*args):  # Accept any arguments passed by events
        canvas.configure(scrollregion=canvas.bbox("all"))

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)
    scrollable_frame.bind("<Configure>", update_scrollregion)

    # Add heading labels for IDs and Names
    id_label = tk.Label(scrollable_frame, text="ID", font=("Arial", 12, "bold"))
    name_label = tk.Label(scrollable_frame, text="Name", font=("Arial", 12, "bold"))
    time_label = tk.Label(scrollable_frame, text="Time", font=("Arial", 12, "bold"))
    id_label.grid(row=0, column=0, sticky="w", padx=10)
    name_label.grid(row=0, column=1, sticky="w", padx=10)
    time_label.grid(row=0, column=2, sticky="w", padx=10)

    for index, row in df.iterrows():
        id_label = tk.Label(scrollable_frame, text=f"{row['ID']}")
        name_label = tk.Label(scrollable_frame, text=f"{row['Name']}")
        time_entry = tk.Label(scrollable_frame, text=f"{row[date_today]}")
        id_label.grid(row=index+1, column=0, sticky="w", padx=10)
        name_label.grid(row=index+1, column=1, sticky="w", padx=10)
        time_entry.grid(row=index+1, column=2, sticky="w", padx=10)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    root.bind("<exclam>", lambda event: root.destroy())

    root.mainloop()
display_attendance()

In [9]:
df = pd.read_excel('students.xlsx')
print(df)

         Name   ID                      2024-05-08  \
0      Ashish   14  15:30:52,  15:30:56,  15:31:02   
1      Chetan   20                             NaN   
2     Lakshmi   34                             NaN   
3    Muzammil   49  16:56:53,  17:00:27,  17:00:32   
4     NIthish   54                             NaN   
5  Pavankumar   59                             NaN   
6     Sanjana   78                             NaN   
7     Shivani   83                             NaN   
8       Suhas   89                        16:56:58   
9        Umar  405                             NaN   

                                  2024-05-09  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3  17:25:17,  17:25:21,  17:25:26,  17:25:31   
4                                        NaN   
5                                        NaN   
6                                        NaN   
7                    

In [14]:
date_today = datetime.now().strftime("%Y-%m-%d")
df2 = df[["Name","ID",date_today]]
df2

,Name,ID,2024-05-11
0,Ashish,14,NaN
1,Chetan,20,NaN
2,Lakshmi,34,NaN
3,Muzammil,49,"08:07:34, 08:07:39, 08:07:44, 08:07:49"
4,NIthish,54,NaN
5,Pavankumar,59,NaN
6,Sanjana,78,NaN
7,Shivani,83,NaN
8,Suhas,89,NaN
9,Umar,405,NaN


In [24]:
import pandas as pd

df1 = pd.read_excel("names_ids.xlsx")
df2 = pd.read_excel("students.xlsx")
print(df1)

           Name   ID
0        Ashish   14
1        Chetan   20
2        Kushal   33
3       Lakshmi   34
4      Muzammil   49
5       NIthish   54
6    Pavankumar   59
7       Sanjana   78
8       Shamant   80
9       Shivani   83
10        Suhas   89
11  Sunitha mam    1
12         Umar  405


In [25]:
print(df2)

           Name   ID                      2024-05-08  \
0        Ashish   14  15:30:52,  15:30:56,  15:31:02   
1        Chetan   20                             NaN   
2       Lakshmi   34                             NaN   
3      Muzammil   49  16:56:53,  17:00:27,  17:00:32   
4       NIthish   54                             NaN   
5       Shivani   83                             NaN   
6         Suhas   89                        16:56:58   
7          Umar  405                             NaN   
8        Kushal   33                             NaN   
9   Sunitha mam    1                             NaN   
10      Shamant   80                             NaN   

                                   2024-05-09  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3   17:25:17,  17:25:21,  17:25:26,  17:25:31   
4                                         NaN   
5                                

In [26]:
print(df2)

           Name   ID                      2024-05-08  \
0        Ashish   14  15:30:52,  15:30:56,  15:31:02   
1        Chetan   20                             NaN   
2       Lakshmi   34                             NaN   
3      Muzammil   49  16:56:53,  17:00:27,  17:00:32   
4       NIthish   54                             NaN   
5       Shivani   83                             NaN   
6         Suhas   89                        16:56:58   
7          Umar  405                             NaN   
8        Kushal   33                             NaN   
9   Sunitha mam    1                             NaN   
10      Shamant   80                             NaN   

                                   2024-05-09  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3   17:25:17,  17:25:21,  17:25:26,  17:25:31   
4                                         NaN   
5                                

In [27]:
missing_ids = df1['ID'].loc[~df1['ID'].isin(df2['ID'])].tolist()

df2 = pd.concat([df2, df1.loc[df1['ID'].isin(missing_ids)]], ignore_index=True)

In [28]:
print(df2)

           Name   ID                      2024-05-08  \
0        Ashish   14  15:30:52,  15:30:56,  15:31:02   
1        Chetan   20                             NaN   
2       Lakshmi   34                             NaN   
3      Muzammil   49  16:56:53,  17:00:27,  17:00:32   
4       NIthish   54                             NaN   
5       Shivani   83                             NaN   
6         Suhas   89                        16:56:58   
7          Umar  405                             NaN   
8        Kushal   33                             NaN   
9   Sunitha mam    1                             NaN   
10      Shamant   80                             NaN   
11   Pavankumar   59                             NaN   
12      Sanjana   78                             NaN   

                                   2024-05-09  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3   17:25:17,  17:2

In [29]:
def train():
	print("training part initiated !")

	recog = cv2.face.LBPHFaceRecognizer_create()

	dataset = 'sizeResized600'

	paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]

	faces = []
	ids = []
	  
	for path in paths:
		# Assuming the filename format is "name-id-count-etc.jpg"
		parts = path.split(os.sep)[-1].split('_')  # os.sep is platform-independent
		label = parts[1]  # ID is now in the second position
		ids.append(int(label))
		faces.append(cv2.imread(path, 0))


	recog.train(faces, np.array(ids))
	recog.save('model600.yml')
 
	#creating excel file to store names and ids
	paths = [os.path.join(dataset, im) for im in os.listdir(dataset)]
	names = []
	ids = []
	for path in paths:
		# Assuming the filename format is "name-id-count-etc.jpg"
		parts = path.split(os.sep)[-1].split('_')
		ids.append(int(parts[1]))  # ID is now in the second position
		names.append(parts[0])  # Name is still in the first position


	# print(names,ids,sep="\n\n")
	data = {'Name': names, 'ID': ids}
	df = pd.DataFrame(data)
	df.drop_duplicates(keep='last',inplace=True)

	# Save the DataFrame to an Excel file
	df.to_excel('names_ids.xlsx', index=False)


	#appending new student data to students excel file
	df1 = pd.read_excel("names_ids.xlsx")
	df2 = pd.read_excel("students.xlsx")
	missing_ids = df1['ID'].loc[~df1['ID'].isin(df2['ID'])].tolist()
	df2 = pd.concat([df2, df1.loc[df1['ID'].isin(missing_ids)]], ignore_index=True)
	df2.to_excel("students.xlsx")
	return
train()

training part initiated !


NameError: name 'cv2' is not defined